# BART 모델 학습 실습
- config -> yaml
- Preprocess class 생성: 데이터 전처리 -> 필요 column 추출, encoder 및 decoder에 적절한 형식으로 변환
- Dataset class 생성
- dataset 생성 함수
- metrics 계산 함수
- trainer 호출 함수
- tokenizer, model 호출 함수
- main 함수: 위 함수 및 클래스들을 사용해서 모델 및 데이터를 호출 및 전처리하여 실제 학습 진행

In [4]:
import pandas as pd
import numpy as np
import os
import random
import yaml
from glob import glob
from pprint import pprint

import torch
import pytorch_lightning as pl
from rouge import Rouge

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb

# 1. Config -> yaml
- 학습 및 추론 등에 필요한 정보들을 파일로 생성한다.
- 이후 해당 파일을 확인하면 어떤 모델을 어떻게 학습했는지 확인할 수 있다.

In [ ]:
# 사용할 BART 모델에 맞는 tokenizer 호출
# config 정의시 tokenizer가 필요해서 먼저 호출
tokenizer = AutoTokenizer.from_pretrained('EbanLee/kobart-summary-v3')

In [ ]:
# 추가로 정의한 사람이름 토큰
name_tokens = [
        '조너스', '블랙', '해머', '양양', '프랭크', '리처드', '레이', 
        '슷클리프', '앨리스', '포드', '윌슨', '라이언', '잭',
        '리사', '베이츠', '하디', '메이', '에이미', '레밍턴', '윌리엄스', 
        '모니카', '브리그스', '케이트', '탕', '사라', '메리', '제임스', 
        '그렉', '토마스', '로라', '크로프트', '원더', '처치', '헨리', 
        '짐씨', '지미', '우드하우스', '린다', '선', '홍', '마리아', 
        '톰', '주디', '한슨', '카론', '브리지스', '로저', '행크스', '코발스키', '존스', 
        '게일', '릭', '산체스', '쿠퍼', '잭슨', '브론', '하오', '휴즈', '자넷', 
        '채양', '이킨스', '화이트', '앤더슨', '양', '리브', '맥퀼린', '스티븐스', 
        '마샤', '문', '조지', '힐튼', '밀러', '마', '찰스', '셀리나', '넬슨', '레이놀즈', '케이', '케이티',
        '저스틴', '서머필드', '모리스', '파커', '파밍턴', '쉘리', '데이비스', '그웬트', '스미스', '와스', 
        '토니', '스튜어트', '셜리', '파슨스', '멜린다', '미셸', '마틴', '루이스', '존슨', '벤슨', '베티', '올리비아', 
        '줄리', '샌들스', '헨더슨', '제이콥', '심슨', '테일러', '트렌트', '루시', '슐러', '제이슨', '셀러즈', 
        '클라크', '월튼', 'Cathy', '제리씨', '필립스', '엘리자베스', '브룩', '아론']

In [ ]:
config_data = {
    # 데이터 경로 등에 대해 설정
    "general": {
        "data_path": "data경로", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "EbanLee/kobart-summary-v3", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    # tokenizer에 대해 설정
    "tokenizer": {
        "encoder_max_len": 512, # encoder 입력 문장의 최대 길이
        "decoder_max_len": 100, # decoder가 생성하는 문장의 최대 길이
        "bos_token": f"{tokenizer.bos_token}", # <bos> token
        "eos_token": f"{tokenizer.eos_token}", # <eos> token
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']+name_tokens
    },
    # 학습시 필요한 설정
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 100, # epochs 횟수
        "learning_rate": 1e-5, # lr
        "per_device_train_batch_size": 50, # train dataloader의 batch size
        "per_device_eval_batch_size": 32, # valid dataloader의 batch size
        "warmup_ratio": 0.1, # learning rate를 점차 변경시키는 정도
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine', # learning rate schedular
        "optim": 'adamw_torch', # optimizer
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True, # 마지막 학습 후 제일 좋은 모델 로드
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 10,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    #"wandb": {
    #    "entity": "wandb_repo",
    #    "project": "project_name",
    #    "name": "run_name"
    #},
}

In [ ]:
# config 파일 생성 -> yaml
config_path = './config.yaml' # 경로
with open(config_path, 'w') as f:
    yaml.dump(config_data, f, allow_unicode=True)

# config 파일 읽어오기
config_path = "./config.yaml"

with open(config_path, "r") as f:
    loaded_config = yaml.safe_load(f)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

# Preprocess class
- 데이터 전처리를 수행한다.
- 필요한 column값 추출
- encoder, decoder에 적합한 형태로 변환

In [5]:
class Preprocess:
    def __init__(self,
                 bos_token:str,
                 eos_token:str) -> None:
        
        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # df에서 필요한 col추출
    def make_set_as_df(self, file_path, is_train=True):
        df=pd.read_csv(file_path)
        if is_train:
            train_df = df[['fname', 'dialogue', 'summary']] # 필요한 col만 가져온다.
            return train_df
        else :
            test_df = df[['fname', 'dialogue']] # 필요한 col만 가져온다.
            return test_df
    
    # BART 모델의 입력, 출력 형태를 맞추기 위한 전처리 
    def make_input(self, dataset, is_test=False):
        if is_test:
            encoder_input = dataset['dialogue'] # 입력 문장
            decoder_input = [self.bos_token] * len(dataset['dalogue']) # decoder는 <bos> token이 들어오면 문장 생성 시작 -> 테스트 문장 개수 만큼 <bos> token 생성
            return encoder_input.tolist(), list(decoder_input)
        
        else :
            encoder_input = dataset['dialogue']
            decoder_input = dataset['dialogue'].apply(lambda x: self.bos_token + str(x)) # 입력 문장의 가장 앞에 <bos> token을 추가하는 작업 ex) '안녕하세요. 반가워요' -> '<bos> 안녕하세요. 반가워요'
            decoder_output = dataset['summary'].apply(lambda x: str(x)+self.eos_token) # 입력 문장의 가장 끝에 <eos> token을 추가하는 작업 ex) '안녕하세요. 반가워요' -> '안녕하세요. 반가워요 <eos>'
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

# Dataset Class
- 학습 및 추론에 사용할 CustomDataset class

In [ ]:
# 학습용 dataset
class DatasetForTrain(torch.utils.data.Dataset):
    

# dataset 생성 함수
- 데이터셋을 생성해주는 함수

# metrics 계산 함수
- 손실함수 정의

# trainer 호출 함수
- 학습에 필요한 trainer 정의
- epochs, batch 등이 정의된 config파일, callback등을 통해 학습을 어떻게 수행할지 정의한다.

# tokenizer, model 호출 함수
- tokenizer와 model을 가져온다.

# main 함수
- 실제 학습 진행